<a href="https://colab.research.google.com/github/Alejandro405/tfg/blob/main/Aprendizaje_Federado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "flwr[simulation]==1.8.0,<2.0.0" --quiet

import platform

if platform.machine() == "arm64":
    !pip install "tensorflow-macos>=2.9.1,!=2.11.1" --quiet
elif platform.machine() == "x86_64":
    !pip install "tensorflow-cpu>=2.9.1,!=2.11.1" --quiet

!pip install pipdeptree --quiet
!pip install "sklearn==1.4.2"  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 750.8 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.11 requires pydantic>=2.7.0, but you have pydantic 1.10.17 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pipdeptree -p 'scikit-learn'

Warning!!! Duplicate package metadata found:
"/usr/lib/python3/dist-packages"
  six                              1.16.0           (using 1.16.0, "/usr/local/lib/python3.10/dist-packages")
NOTE: This warning isn't a failure warning.
------------------------------------------------------------------------
Warning!!! Possibly conflicting dependencies found:
* cudf-cu12==24.4.1
 - pandas [required: >=2.0,<2.2.2dev0, installed: 1.5.3]
* google-colab==1.0.0
 - pandas [required: ==2.0.3, installed: 1.5.3]
* orbax-checkpoint==0.4.4
 - jax [required: >=0.4.9, installed: 0.3.25]
* pandas-gbq==0.19.2
 - google-auth-oauthlib [required: >=0.7.0, installed: 0.4.6]
* PyGObject==3.42.1
 - pycairo [required: >=1.16.0, installed: ?]
* tensorflow==2.12.0
 - keras [required: >=2.12.0,<2.13, installed: 3.3.3]
 - tensorboard [required: >=2.12,<2.13, installed: 2.16.2]
* tensorflow-metadata==1.15.0
 - protobuf [required: >=3.20.3,<4.21, installed: 4.25.3]
* tf_keras==2.15.1
 - tensorflow [required: >=2.15,<2

In [ ]:
import os
import sys
import json
import argparse

import flwr as fl
import tensorflow as tf
import numpy as np
import pandas as pd

from abc import abstractmethod
from typing import Any, Dict, List, Tuple

from flwr.common import Metrics
from flwr.simulation import start_simulation

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


from typing import Dict, List, Tuple
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from tensorflow.keras import layers, models


dataset_name = 'cse-cic'

BATCH_SIZE = 64

NUM_CLIENTS = 10
NUM_ROUNDS = 50
VERBOSE = 0

SEED = 46# 47

# Reproducibilidad de los experimentos
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Dataset



In [ ]:
class PandasPartitioner:
    def __init__(self, data: pd.DataFrame, numClients: int):
        self.data = data
        self.numClients = numClients
        self.partitions = {
            'train': {f'{cid}': None for cid in range(numClients)},
            'validation': None
        }

        self.build_partitions()

    # Declarar un método estático que genere conjuntos de datos de entrenamiento y test usando sklearn
    @staticmethod
    def split_data(data: pd.DataFrame, x_columns: List[str], test_size: float, random_state: int) -> tuple:
        '''
        Divide un dataframe de pandas en dos subconjuntos de entrenamiento y test.

        :param data: dataframe de pandas a dividir
        :param x_columns: lista con los nombres de las columnas que se usarán como variables independientes
        :param test_size: tamaño del conjunto de test
        :param random_state: semilla para la generación de números aleatorios
        :return: tupla con los conjuntos de entrenamiento y test
        '''
        X = data[x_columns]
        y = data.drop(columns=x_columns)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        return X_train, X_test, y_train, y_test

    # Declarar un método estático que convierte un dataframe pandas en un numpy array
    @staticmethod
    def pandas_to_numpy(data: pd.DataFrame) -> np.ndarray:
        '''
        Convierte un dataframe de pandas en un numpy array. Importatnte tener encuenta que el dataframe no debe tener columnas de tipo string solo numéricas.

        :param data: dataframe de pandas a convertir
        :return:  numpy array con los datos del dataframe
        '''
        return data.to_numpy()

    def get_split(self, split: str) -> pd.DataFrame:
        return self.partitions[split]

    def get_partition(self, split:str, partition_number: int) -> pd.DataFrame:
        return self.partitions[split][f'{partition_number}']

    def get_partitions(self):
        return self.partitions

    def build_partitions(self):
        train_size = int(len(self.data) * 0.8)
        test_size = len(self.data) - train_size

        # Dividir el dataset en train y test de manera aleatoria por índices
        train = self.data.sample(frac=0.8, random_state=0)
        test = self.data.drop(train.index)

        self.partitions['validation'] = test

        # Dividir el dataset de train en las particiones de los clientes, tantos subconjuntos como clientes haya (de manera aleatoria y excluyente)
        for i in range(self.numClients):
            client_data = train.sample(frac=1/self.numClients, random_state=i)

            self.partitions['train'][f'{i}'] = client_data
            train = train.drop(client_data.index)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
raw_df = pd.read_csv(f'./drive/MyDrive/{dataset_name}/results/escalado_ecodeado_ces_cic.csv')

raw_df.replace([np.inf, -np.inf], np.nan, inplace=True)
raw_df.dropna(inplace=True)
raw_df.reset_index(inplace=True)

X_COLUMNS = ['Flow Pkts/s', 'Fwd Pkt Len Mean', 'Tot Fwd Pkts', 'Init Fwd Win Byts',
       'Fwd Seg Size Avg', 'Subflow Fwd Pkts', 'ACK Flag Cnt',
       'Subflow Fwd Byts', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Bwd IAT Min',
       'Fwd Header Len', 'Bwd IAT Mean']
Y_COLUMNS = ['Benign', 'Bot', 'Brute Force -Web', 'Brute Force -XSS',
       'DDOS attack-HOIC', 'DDOS attack-LOIC-UDP', 'DoS attacks-GoldenEye',
       'DoS attacks-Hulk', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Slowloris',
       'FTP-BruteForce', 'Infilteration', 'SQL Injection', 'SSH-Bruteforce']

num_clases = len(Y_COLUMNS)

df = raw_df.drop('index', axis=1)

partitioner = PandasPartitioner(df, NUM_CLIENTS)

In [ ]:
raw_df = pd.read_csv(f'./drive/MyDrive/{dataset_name}/results/escalado_ecodeado_nsl_kdd.csv')

raw_df.replace([np.inf, -np.inf], np.nan, inplace=True)
raw_df.dropna(inplace=True)
raw_df.reset_index(inplace=True)

df = raw_df.copy()

In [ ]:
raw_df = pd.read_csv(f'./drive/MyDrive/{dataset_name}/results/escalado_ecodeado_unsw_nb15.csv')

raw_df.replace([np.inf, -np.inf], np.nan, inplace=True)
raw_df.dropna(inplace=True)
raw_df.reset_index(inplace=True)

df = raw_df.copy()

# Modelo Federado

In [ ]:
#UNSW NB15
unsw_nb15_hparams = {
    'learning_rate': 0.0001,
    'kernel_initializer': 'lecun_uniform',
    'l2_regularization': 0.1,
    'activations': 'elu',
    'encoder_structure': [100, 10, 50, 10],
    'classifier': [60, 60, 20, 70, 10]

}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# NSL KDD
nsl_kdd_hparams = {'learning_rate': 1e-06,
 'kernel_initializer': 'glorot_uniform',
 'l2_regularization': 0.0001,
 'activations': 'relu',
 'num_hidden_layers': 5,
  'encoder': [10, 60, 100, 40, 100],
  'classifier': [30, 40, 70, 30, 20]
 }


In [ ]:
# CSE CIC IDS 2018
cse_cic_hparams = {
    'learning_rate': 0.01,
    'kernel_initializer': 'he_normal',
    'l2_regularization': 0.001,
    'activations': 'elu',
    'encoder': [60, 40, 30, 20, 10],
    'classifier': [80, 40, 90, 30, 30]
}

In [ ]:
class Classifier:
    @abstractmethod
    def train(self, x_data: np.ndarray, y_data:np.ndarray, epochs, batch_size) -> Dict[str, Any]:
        pass

    @abstractmethod
    def predict(self, x_data: np.ndarray) -> np.ndarray:
        pass

    @abstractmethod
    def evaluate(self, y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, Any]:
        pass

    @abstractmethod
    def compile(self, optimizer: str, loss: str, metrics: List[str]) -> None:
        pass

from sklearn.svm import SVC

class SVMClassifier(Classifier):
    def __init__(self):
        self.model = SVC(gamma='auto')

    def train(self, x_data: np.ndarray, y_data: np.ndarray, epochs, batch_size) -> Dict[str, Any]:
        self.model.fit(x_data, y_data)

        return {}

    def predict(self, x_data: np.ndarray) -> np.ndarray:
        return self.model.predict(x_data)

    def evaluate(self, y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, Any]:
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }

    def compile(self, optimizer: str, loss: str, metrics: List[str]) -> None:
        # Implementa un proceso de compilación para el modelo basado en SVM
        pass

In [ ]:
class ClassifierWrapper(tf.keras.Model):
    def __init__(self, classifier: Classifier):
        super(ClassifierWrapper, self).__init__()
        self.sklearn_classifier: Classifier = classifier
        # Añade estructura de datos para almacenar las metricas de evaluación
        self.metrics_tracker = {
            'accuracy': tf.keras.metrics.Accuracy(),
            'precision': tf.keras.metrics.Precision(),
            'recall': tf.keras.metrics.Recall(),
            'f1_score': tf.keras.metrics.Mean()
        }

    def compile(self, optimizer: str, loss: str, metrics: List[str]) -> None:
        super(ClassifierWrapper, self).compile()
        self.optimizer = tf.keras.optimizers.get(optimizer)
        self.loss_fn = tf.keras.losses.get(loss)

    def fit(self, x_train: np.ndarray, y_train: np.ndarray, epochs: int = 1, batch_size: int = 32,
            validation_data: Tuple[np.ndarray, np.ndarray] = None) -> None:
        # Si el clasificador es una red neuronal profunda, se debe entrenar con Keras y ha de ser compilado
        if isinstance(self.sklearn_classifier, tf.keras.Model):
            self.sklearn_classifier.compile(optimizer=self.optimizer, loss=self.loss_fn, metrics=self.metrics)

            self.sklearn_classifier.train(
                x_train,
                y_train,
                epochs=epochs,
                batch_size=batch_size
            )
        self.sklearn_classifier.train(
            x_train,
            y_train
        )

        # Si hay datos de validación, evaluar el modelo
        metrics = None
        if validation_data:
            x_val, y_val = validation_data
            metrics = self.evaluate(x_val, y_val)

            # Pasar los valores del diccionario a tupla de tantoas componetes como claves haya
            metrics = tuple(metrics.values())

        return metrics

    def call(self, inputs: tf.Tensor, training: bool) -> tf.Tensor:
        inputs_np = inputs.numpy()
        predictions = self.classifier.predict(inputs_np)
        return tf.convert_to_tensor(predictions, dtype=tf.float32)

    def evaluate(self, x_data: np.ndarray, y_data: np.ndarray, **kwargs) -> Dict[str, Any]:
        y_pred =self.sklearn_classifier.predict(x_data)
        results = self.sklearn_classifier.evaluate(y_data, y_pred)

        return results

    @property
    def metrics(self) -> Any:
        return self.metrics_tracker.values()

In [ ]:
def build_autoencoder(hparams):
    # Encoder
    input_layer = Input(shape=(len(X_COLUMNS),))
    encoded = input_layer

    for i, units in enumerate(hparams['encoder']):
        encoded = Dense(units, activation=hparams['activations'], kernel_initializer=hparams['kernel_initializer'],
                        kernel_regularizer=l2(hparams['l2_regularization']))(encoded)

    encoder = models.Model(input_layer, encoded)

    # Decoder
    decoded = encoded

    decoder_units = reversed(hparams['encoder'][:-1])
    for i, units in enumerate(hparams['classifier']):
        decoded = Dense(units, activation=hparams['activations'], kernel_initializer=hparams['kernel_initializer'],
                        kernel_regularizer=l2(hparams['l2_regularization']))(decoded)

    output_layer = Dense(len(X_COLUMNS), activation='softmax')(decoded)

    decoder = models.Model(encoded, output_layer)

    # Autoencoder
    autoencoder = Model(input_layer, output_layer)

    # Compile the model
    autoencoder.compile(optimizer=Adam(learning_rate=hparams['learning_rate']),
                        loss='mse',
                        metrics=['accuracy'])
    return autoencoder, encoder, decoder

autoencoder, encoder, decoder = build_autoencoder(cse_cic_hparams)

In [ ]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import layers, losses

class AutoencoderClassifier:
    def __init__(self, input_shape, hparams, classifier: ClassifierWrapper=None, nClasses: int=1):
        self.learning_rate = hparams['learning_rate']
        self.kernel_init = hparams['kernel_initializer']
        self.l2_value = hparams['l2_regularization']
        self.activation = hparams['activations']
        self.numClasses = nClasses
        self.input_shape = input_shape
        self.hparams = hparams

        #aut_cod, encoder, decoder = self.build_autoencoder(hparams)

        self.autoencoder  = autoencoder
        self.__encoder = encoder
        self.__decoder = decoder

        #self.__encoder = self.build_encoder(input_shape, hparams['encoder'])
        #self.__decoder = self.build_decoder(hparams['encoder'])

        #self.autoencoder = models.Model(self.__encoder.input, self.__decoder(self.__encoder.output))

        self.__classifier = classifier or self.build_classifier(hparams['classifier'], input_dim=hparams['encoder'][-1], num_classes=nClasses)

    def build_autoencoder(hparams):
        # Encoder
        input_layer = Input(shape=(len(X_COLUMNS),))
        encoded = input_layer

        for i, units in enumerate(hparams['encoder']):
            encoded = Dense(units, activation=hparams['activations'], kernel_initializer=hparams['kernel_initializer'],
                            kernel_regularizer=l2(hparams['l2_regularization']))(encoded)

        encoder = models.Model(input_layer, encoded, 'encoder')

        # Decoder
        decoded = encoded

        decoder_units = reversed(hparams['encoder'][:-1])
        for i, units in enumerate(hparams['classifier']):
            decoded = Dense(units, activation=hparams['activations'], kernel_initializer=hparams['kernel_initializer'],
                            kernel_regularizer=l2(hparams['l2_regularization']))(decoded)

        output_layer = Dense(len(X_COLUMNS), activation='softmax')(decoded)

        decoder = models.Model(encoded, output_layer, 'decoder')

        # Autoencoder
        autoencoder = Model(input_layer, output_layer, 'autoencoder')

        # Compile the model
        autoencoder.compile(optimizer=Adam(learning_rate=hparams['learning_rate']),
                            loss='mse',
                            metrics=['accuracy'])
        return autoencoder, encoder, decoder

    def build_encoder(self, input_shape, structure):
        inputs = layers.Input(shape=input_shape)
        x = inputs
        for units in structure:
            x = layers.Dense(
                units,
                activation=self.activation,
                kernel_initializer=self.kernel_init,
                kernel_regularizer=tf.keras.regularizers.l2(self.l2_value)
            )(x)

        return models.Model(inputs, x, name='encoder')

    def build_decoder(self, structure):
        latent_dim = structure[-1]
        inputs = layers.Input(shape=(latent_dim,))

        x = inputs
        decoder_units = reversed(structure[:-1])
        for units in decoder_units:
            x = layers.Dense(
                units,
                activation='relu',
                kernel_initializer=self.kernel_init,
                kernel_regularizer=tf.keras.regularizers.l2(self.l2_value)
            )(x)
        x = layers.Dense(self.input_shape[0], activation='sigmoid')(x)

        return models.Model(inputs, x, name='decoder')

    def build_classifier(self, params, input_dim=None, num_classes=1):
        inputs = layers.Input(shape=(input_dim,))

        x = inputs
        for units in params:
            x = layers.Dense(
                units=units,
                activation=self.activation,
                kernel_initializer=self.kernel_init,
                kernel_regularizer=tf.keras.regularizers.l2(self.l2_value)
            )(x)

        # Añadir la capa de salida de tipo OutputLayer con activación sigmoide para las nClases
        outputs = layers.Dense(num_classes, activation='sigmoid')(x)

        return models.Model(inputs, outputs, name='classifier')

    def compile(self, optimizer='adagrad', loss='binary_crossentropy'):
        self.__encoder.compile(optimizer=optimizer)
        self.__decoder.compile(optimizer=optimizer)
        self.autoencoder.compile(optimizer=optimizer, loss=losses.MeanSquaredError())
        self.__classifier.compile(optimizer=optimizer, loss=losses.MeanSquaredError(), metrics=['accuracy', Precision(), Recall()])

    def train(self, x_train: np.ndarray, y_train: np.ndarray, epochs: int =2, batch_size:int =32):
        input = x_train.copy()
        history = self.train_autoencoder(input, epochs=epochs, batch_size=batch_size)
        encoded_features = self.__encoder.predict(input)
        self.train_classifier(encoded_features, y_train, epochs=epochs, batch_size=batch_size)

        return history

    def train_autoencoder(self, x_train: np.ndarray, epochs=5, batch_size=16):
        self.autoencoder.compile(optimizer='adagrad', loss=losses.MeanSquaredError())
        history = self.autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

        print(history.history)

        return history

    def train_classifier(self, x_train: np.ndarray, y_train: np.ndarray, epochs=2, batch_size=32):

        self.__classifier.fit(x_train, y_train)

    def evaluate(self, x_test: np.ndarray, y_test: np.ndarray):
        input = x_test.copy()

        encoded_features = self.__encoder(input)
        encoded_features = np.array(encoded_features)
        results = self.__classifier.evaluate(encoded_features, y_test)

        return results

    def predict(self, x_data: np.ndarray):
        y = self.__encoder.predict(x_data)
        y = np.array(y)

        return self.__classifier.predict(y)

    def get_weights(self):
        return self.autoencoder.get_weights()

    def set_weights(self, weights):
        self.autoencoder.set_weights(weights)


def init_model(in_dim, nClasses, hparams):
    return AutoencoderClassifier((in_dim, ), hparams, None, nClasses)

In [ ]:
def switch_case(value):
    return {
        'unsw-nb15': unsw_nb15_hparams,
        'cse-cic': cse_cic_hparams,
        'nsl-kdd': nsl_kdd_hparams
    }.get(value)


hparams = switch_case(dataset_name)

In [ ]:
model = init_model(len(X_COLUMNS), len(Y_COLUMNS), cse_cic_hparams)

In [ ]:
from typing import List
import datetime


from flwr.common import (
    ndarrays_to_parameters,
    parameters_to_ndarrays,
    Status,
    Code,
    GetParametersIns,
    EvaluateIns,
    EvaluateRes,
    GetParametersRes,
    GetPropertiesIns,
    GetPropertiesRes,
    FitIns,
    FitRes,
    Parameters
)


log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

def client_factory(partitioner: PandasPartitioner):
    def client_fn(cid: str) -> fl.client.Client:
        """Construye un cliente a partir del id especificado como parámetro. Al cliente se le asignará los dataset de
        entrenamieno y validación"""

        # Create and return client
        return TensorFlowClient(
              init_model(
                  len(X_COLUMNS),
                  len(Y_COLUMNS),
                  hparams
              ), partitioner.get_partition('train', int(cid))
          ).to_client()

    return client_fn


class TensorFlowClient(fl.client.Client):
    def __init__(self, model: AutoencoderClassifier, data: pd.DataFrame):
        self.model = model
        self.data = data
        self.history = None

        self.x_train, self.x_test, self.y_train, self.y_test = PandasPartitioner.split_data(data, X_COLUMNS, 0.2, 0)

    def get_parameters(self, GetParametersIns) -> fl.common.GetParametersRes:
        """Return the current model parameters."""
        return GetParametersRes(
            status=Status(
                code=Code.OK,
                message="El cliente serializó el modelo"
            ),
            parameters=ndarrays_to_parameters(self.model.get_weights())
        )

    def fit(self, fitinstr: fl.common.FitIns) -> fl.common.FitRes:

        """Train parameters on the locally held training set."""

        parameters, config = parameters_to_ndarrays(fitinstr.parameters), fitinstr.config

        # Update local model parameters. Sets the values of layer.weights from a list of numpy arrays.
        self.model.set_weights(parameters)
        self.model.compile()

        # Get hyperparameters for this round
        batch_fit: int = config["batch_size"]
        epochs_fit: int = config["local_epochs"]


        history = self.model.train_autoencoder(
            x_train=self.x_train,
            batch_size=batch_fit,
            epochs=epochs_fit
        )

        """
        history = self.model.train(
            x_train=self.x_train,
            y_train=self.y_train,
            epochs=epochs_fit,
            batch_size=batch_fit
        )"""

        # Return updated model parameters and results
        parameters_prime = self.model.get_weights()

        num_examples_train = len(self.x_train)

        results = {
            key: np.mean(history.history[key]) if key in ['loss', 'val_loss'] else history.history[key]
            for key in history.history.keys()
        }

        return FitRes(
            status=Status(
                code=Code.OK,
                message="El cliente ha entrenado el modelo"
            ),
            parameters=ndarrays_to_parameters(parameters_prime),
            num_examples=num_examples_train,
            metrics=results
        )

    def evaluate(self, evalInstr: EvaluateIns) -> EvaluateRes:
        """Evaluate parameters on the locally held test set.

        parameters, config = evalInstr.parameters, evalInstr.config
        self.model.compile()

        # Get config values
        steps: int = config["val_steps"]

        # Evaluate global model parameters on the local test data and return results
        evalResult = self.model.evaluate(self.x_test, self.y_test)
        num_examples_test = len(self.x_test)"""
        return EvaluateRes(
            status=Status(
                code=Code.EVALUATE_NOT_IMPLEMENTED,
                message="La evaluación no se ha de hacer en el cliente. Esto corre acuenta del server"
            ),
            metrics={},
            loss=0.0,
            num_examples=0,
        )

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        self.model.set_weights(weights)

# Clientes

# Servidor

In [ ]:
from typing import Optional

from tensorflow.keras.metrics import Precision, Recall

model = init_model(len(X_COLUMNS), len(Y_COLUMNS), hparams)

def get_evaluate_fn(refModel, partitioner: PandasPartitioner):
    """Return an evaluation function for server-side evaluation."""
    eval = partitioner.get_split("validation")
    x_train, x_test, y_train, y_test = PandasPartitioner.split_data(eval, X_COLUMNS, 0.33, 42)
    #x_test, y_test = eval[X_COLUMNS], eval[Y_COLUMNS]

    # The `evaluate` function will be called after every round
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        print('[INFO]   Toca Evaluar el Modelo en el Servidor')
        refModel.set_weights(parameters)  # Update model with the latest parameters
        refModel.compile()
        refModel.train(x_train, y_train, epochs=1, batch_size=256)
        loss, acc, prec, rec = refModel.evaluate(x_test, y_test)

        #return metrics
        return loss, {
            'accuracy': acc,
            'precission':prec,
            'racall':rec
        }

    return evaluate


def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Keep batch size fixed at 32, perform two rounds of training with one local epoch,
    increase to two local epochs afterwards.
    """
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": 2 if server_round < 2 else 5,
    }
    return config


def evaluate_config(server_round: int):
    """Return evaluation configuration dict for each round.

    Perform five local evaluation steps on each client (i.e., use five batches) during
    rounds one to three, then increase to ten local evaluation steps.
    """
    val_steps = 2 if server_round < 4 else 5
    return {"val_steps": val_steps}

from typing import Tuple, Union
from flwr.common import Scalar
from flwr.server.client_proxy import ClientProxy

class FedCS(fl.server.strategy.FedAvg):
  def aggregate_fit(self, server_round: int, results: List[Tuple[ClientProxy, FitRes]], failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]]) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
      for failure in failures:
        if isinstance(failure, tuple):
            client_proxy, fit_res = failure
            print(f"[INFO]   ClientProxy: {client_proxy}, FitRes: {fit_res}")
        elif isinstance(failure, BaseException):
            print(f"[INFO]   Excepción: {type(failure).__name__}, Mensaje: {str(failure)}")

      return super().aggregate_fit(server_round, results, failures)

# Simulación

## Tensorboard

In [ ]:
# Cargar el módulo TensorBoard notebook para ejecutarlo dentro de Colab
%load_ext tensorboard
%tensorboard --logdir $log_dir

In [ ]:
import logging

# Deshabilitar todos los manejadores de logging globales de Python
logging.getLogger().handlers = []

# Configurar el logging específico para Flower
fl.common.logger.configure("[INFO]")

In [ ]:
#strategy = fl.server.strategy.FedAvg(
strategy = FedCS(
    fraction_fit=1.0,  # Ensure all clients are used for training
    fraction_evaluate=0.0,  # No client-side evaluation
    min_fit_clients=1,
    min_evaluate_clients=1,
    min_available_clients=1,
    evaluate_fn=get_evaluate_fn(model, partitioner),  # Server-side evaluation only
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)

client_fn = client_factory(partitioner)

history = fl.simulation.start_simulation(
    client_fn=client_fn,  # a function that spawns a particular client
    num_clients=NUM_CLIENTS,  # total number of clients
    config=fl.server.ServerConfig(
        num_rounds=NUM_ROUNDS
    ),  # minimal config for the server loop telling the number of rounds in FL
    strategy=strategy,  # our strategy of choice
    client_resources={
        "num_cpus": 2,
        "num_gpus": 0.0,
    },  # (optional) controls the degree of parallelism of your simulation.
    # Lower resources per client allow for more clients to run concurrently
    # (but need to be set taking into account the compute/memory footprint of your run)
    # `num_cpus` is an absolute number (integer) indicating the number of threads a client should be allocated
    # `num_gpus` is a ratio indicating the portion of gpu memory that a client needs.
)

INFO :      Starting Flower simulation, config: num_rounds=50, no round_timeout
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-28 09:17:24,873	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3954160435.0, 'memory': 7908320871.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters


[INFO]   Toca Evaluar el Modelo en el Servidor
428/428 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.9026 - val_loss: 0.8865
{'loss': [0.8981467485427856], 'val_loss': [0.8864668011665344]}
3800/3800 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
3800/3800 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.5588 - loss: 0.6594 - precision: 0.3828 - recall: 0.7361
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8220 - loss: 0.5505 - precision: 0.8223 - recall: 0.8220


INFO :      initial parameters (loss, other metrics): 0.5505301356315613, {'accuracy': 0.8216062784194946, 'precission': 0.8218669891357422, 'racall': 0.8216062784194946}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 25 clients (out of 25)
(ClientAppActor pid=1260) /usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 12 variables whereas the saved optimizer has 2 variables. 
(ClientAppActor pid=1260)   saveable.load_own_variables(weights_store.get(inner_path))
(ClientAppActor pid=1260) /usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 14 variables whereas the saved optimizer has 2 variables. 
(ClientAppActor pid=1260)   saveable.load_own_variables(weights_store.get(inner_path))


(ClientAppActor pid=1260) Epoch 1/2
  1/128 ━━━━━━━━━━━━━━━━━━━━ 3:39 2s/step - loss: 0.9018
 56/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9051
115/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9050
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.9049 - val_loss: 0.9036
(ClientAppActor pid=1260) Epoch 2/2
  1/128 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 0.9001
 35/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9014
 67/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9014
 99/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9013
115/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9011
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.9010 - val_loss: 0.8987
(ClientAppActor pid=1260) {'loss': [0.9040879011154175, 0.899937093257904], 'val_loss': [0.9035628437995911, 0.8986921310424805]}


(ClientAppActor pid=1260) /usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 12 variables whereas the saved optimizer has 2 variables. 
(ClientAppActor pid=1260)   saveable.load_own_variables(weights_store.get(inner_path))
(ClientAppActor pid=1260) /usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 14 variables whereas the saved optimizer has 2 variables. 
(ClientAppActor pid=1260)   saveable.load_own_variables(weights_store.get(inner_path))


...

(ClientAppActor pid=1260) Epoch 1/5
  1/256 ━━━━━━━━━━━━━━━━━━━━ 7:38 2s/step - loss: 0.6906
 57/256 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6910
114/256 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6909
154/256 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6908
185/256 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6908
216/256 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6907
247/256 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6906
256/256 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.6906 - val_loss: 0.6889
(ClientAppActor pid=1260) Epoch 2/5
  1/256 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 0.6911
 36/256 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6886
 71/256 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6887
 93/256 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6887
124/256 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6886
146/256 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6886
179/256 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6885
204/256 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6885
239/256 ━━━

(ClientAppActor pid=1260) /usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 12 variables whereas the saved optimizer has 2 variables. 
(ClientAppActor pid=1260)   saveable.load_own_variables(weights_store.get(inner_path))
(ClientAppActor pid=1260) /usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 14 variables whereas the saved optimizer has 2 variables. 
(ClientAppActor pid=1260)   saveable.load_own_variables(weights_store.get(inner_path))


(ClientAppActor pid=1260) Epoch 1/5
  1/193 ━━━━━━━━━━━━━━━━━━━━ 7:46 2s/step - loss: 0.6905
 55/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6911
110/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6910
144/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6909
173/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6908
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.6908 - val_loss: 0.6892
(ClientAppActor pid=1260) Epoch 2/5
 30/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6890 
 86/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6890
131/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6889
155/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6889
183/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6889
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6889 - val_loss: 0.6876
(ClientAppActor pid=1260) Epoch 3/5
  1/193 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 0.6888
 49/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6877
 76/193 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6877
118/193 ━━━

INFO :      aggregate_fit: received 25 results and 0 failures


193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6842 - val_loss: 0.6826
(ClientAppActor pid=1260) {'loss': [0.6902925372123718, 0.6886312365531921, 0.686975359916687, 0.6853250861167908, 0.683679461479187], 'val_loss': [0.689218282699585, 0.6875600814819336, 0.6859071254730225, 0.6842591762542725, 0.6826167702674866]}
[INFO]   Toca Evaluar el Modelo en el Servidor
428/428 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.6805 - val_loss: 0.6778
{'loss': [0.6795807480812073], 'val_loss': [0.6778499484062195]}
3800/3800 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
3800/3800 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.8232 - loss: 0.2537 - precision_18: 0.8232 - recall_18: 0.8232
1077/1872 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8223 - loss: 0.2458 - precision_18: 0.8223 - recall_18: 0.8223

In [ ]:
loss = history.losses_centralized
acc = history.metrics_centralized['accuracy']
rec = history.metrics_centralized['racall']
pre = history.metrics_centralized['precission']
metrics = [loss, acc, rec, pre]
metric_names = ['loss', 'accuracy', 'recall', 'precission']

In [ ]:
def plot_time_series(data, title='Time Series Plot', x_label='X', y_label='Y'):
    """
    Genera un gráfico de series temporales a partir de una lista de puntos (x, y).

    :param data: Lista de tuplas (x, y)
    :param title: Título del gráfico
    :param x_label: Etiqueta del eje X
    :param y_label: Etiqueta del eje Y
    """
    # Crear un DataFrame a partir de la lista de puntos
    df = pd.DataFrame(data, columns=['x', 'y'])

    # Configurar seaborn
    sns.set(style='whitegrid')

    # Crear el gráfico
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='x', y='y', data=df, marker='o', palette='viridis')

    # Configurar etiquetas y título
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    # Mostrar el gráfico
    plt.show()

In [ ]:
plot_time_series(pre, 'Serie Temporal de precission', x_label='Rounds', y_label='Precision')

In [ ]:
plot_time_series(acc, 'Serie Temporal de accuracy', x_label='Rounds', y_label='Accuracy')

In [ ]:
plot_time_series(loss, 'Serie Temporal de loss', x_label='Rounds', y_label='Loss')

In [ ]:
plot_time_series(rec, 'Serie Temporal de recall', x_label='Rounds', y_label='Recall')

In [ ]:
history

In [ ]:
X = df[X_COLUMNS]
Y = df[Y_COLUMNS]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

del model
model = init_model(len(X_COLUMNS), len(Y_COLUMNS), hparams)

In [ ]:
autoencoder.fit(x_train, x_train, batch_size=512, epochs=3)

Epoch 1/3
1008/1188 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7569 - loss: 0.0929

KeyboardInterrupt: 

In [ ]:
x_train.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(607971, 13)

In [ ]:
len(X_COLUMNS)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


13